# Тестовое задание

In [108]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import string

In [183]:
count_vect = CountVectorizer(max_df = 0.8, min_df = 0.05, ngram_range=(1, 3))
tfidf_vect = TfidfVectorizer(max_df = 0.8, min_df = 0.05, ngram_range=(1, 3))
porter_stemmer = nltk.stem.PorterStemmer()

## Cкачиваем файл с данными

In [4]:
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
#или
#!wget https://drive.google.com/open?id=1DwwTyIn5zW7KTxp4bONHtKNa6DlT9PUj

## Если необходимо, преобразуем JSON к CSV

In [ ]:
df = pd.read_json ('reviews_Electronics_5.json', lines=True )
df.to_csv ('Electronics_5.csv', index = None)

# Предобработка

In [110]:
dat = pd.read_csv("Electronics_5.csv")[["asin","reviewText", "overall"]]

## Добавляем бинрную оценку

In [112]:
dat["bin"] = dat.overall > 3

## Отрежем кусок даты

In [120]:
dat_t, dat = dat, dat[0:11]

## Натравим на необработанные данные

In [ ]:
dat_count_deft = count_vect.fit_transform(dat.reviewText)
dat_tfidf_deft = tfidf_vect.fit_transform(dat.reviewText)


## Приводим к базовой форме

In [121]:
# for i in range(len(dat["words"])):
#     dat["words"][i] = (" ").join(list(map(lambda x: porter_stemmer.stem(x.lower().translate(str.maketrans('', '', string.punctuation))), \
#                                           filter(lambda word: word.lower() not in nltk.corpus.stopwords.words('english'), dat["words"][i]))))

In [187]:
dat["words"] = list(map(lambda s: nltk.word_tokenize(s.replace("'", " ")), dat.reviewText))

for i in range(len(dat.words)):
    dat.words[i] = (" ").join(list(map(lambda s: (porter_stemmer.stem(s)), dat.words[i]))).lower().translate(str.maketrans('', '', string.punctuation))

/home/denis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/denis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/denis/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org

## Натравим на обработанные данные

In [184]:
dat_count = count_vect.fit_transform(dat.words)
dat_tfidf = tfidf_vect.fit_transform(dat.words)

## Проверим, что получилось

In [185]:
dat

,asin,reviewText,overall,bin,words
0,0528881469,We got this GPS for my husband who is an (OTR)...,5,True,we got thi gp for my husband who is an otr o...
1,0528881469,"I'm a professional OTR truck driver, and I bou...",1,False,i m a profession otr truck driver and i bough...
2,0528881469,"Well, what can I say. I've had this unit in m...",3,False,well what can i say i ve had thi unit in my ...
3,0528881469,"Not going to write a long review, even thought...",2,False,not go to write a long review even thought th...
4,0528881469,I've had mine for a year and here's what we go...,1,False,i ve had mine for a year and here s what we go...
5,0594451647,I am using this with a Nook HD+. It works as d...,5,True,i am use thi with a nook hd it work as descri...
6,0594451647,The cable is very wobbly and sometimes disconn...,2,False,the cabl is veri wobbl and sometim disconnect ...
7,0594451647,This adaptor is real easy to setup and use rig...,5,True,thi adaptor is real easi to setup and use righ...
8,0594451647,This adapter easily connects my Nook HD 7&#34;...,4,True,thi adapt easili connect my nook hd 7 34 to...
9,0594451647,This product really works great but I found th...,5,True,thi product realli work great but i found the ...


In [10]:
dat_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [2, 2, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [11]:
dat_tfidf.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.05515774,
        0.        ],
       [0.05199244, 0.06082669, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.07066887, 0.        ,
        0.09400997],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [118]:
dat_count_deft.toarray()

array([[0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 1, 3, 0],
       [0, 2, 2, ..., 0, 5, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 6, 3],
       [0, 0, 0, ..., 0, 0, 0]])

In [154]:
len(dat_count.toarray()[0])

662

In [160]:
len(count_vect.get_feature_names())

662

In [186]:
print(count_vect.get_feature_names())

['10', '10 feet', '10 feet away', '10 ton', '10 ton counti', '10 via', '10 via or', '10 year', '10 year old', '100', '100 page', '100 page long', '100 thorough', '100 thorough list', '12', '12 bridg', '12 bridg when', '13', '13 of', '13 of cours', '13 the', '13 the length', '15', '15 mile', '15 mile down', '1mil', '1mil mile', '1mil mile and', '23', '23 mile', '23 mile of', '2400', '2400 dollar', '2400 dollar ticket', '25', '25 in', '25 in denver', '25ft', '25ft high', '25ft high qualiti', '30', '30 on', '30 on cheap', '34', '34 to', '34 to my', '34 tv', '34 tv is', '3d', '3d from', '3d from few', '45', '45 second', '45 second to', '465t', '465t or', '465t or the', '465t realli', '465t realli do', '50', '50 have', '50 have crash', '52', '52 34', '52 34 tv', '53', '53 and', '53 and the', '700', '700 at', '700 at truck', '700 even', '700 even though', '700 for', '700 for sever', '700 howev', '700 howev want', '700 it', '700 it seem', '700 or', '700 or is', '700 rout', '700 rout me', '700

In [167]:
tok = count_vect.build_analyzer()

In [171]:
tok(dat.reviewText[1])

['professional',
 'otr',
 'truck',
 'driver',
 'and',
 'bought',
 'tnd',
 '700',
 'at',
 'truck',
 'stop',
 'hoping',
 'to',
 'make',
 'my',
 'life',
 'easier',
 'rand',
 'mcnally',
 'are',
 'you',
 'listening',
 'first',
 'thing',
 'did',
 'after',
 'charging',
 'it',
 'was',
 'connect',
 'it',
 'to',
 'my',
 'laptop',
 'and',
 'install',
 'the',
 'software',
 'and',
 'then',
 'attempt',
 'to',
 'update',
 'it',
 'the',
 'software',
 'detected',
 'problem',
 'with',
 'my',
 'update',
 'and',
 'wanted',
 'my',
 'home',
 'address',
 'so',
 'could',
 'be',
 'sent',
 'patch',
 'on',
 'an',
 'sd',
 'card',
 'hello',
 'don',
 'think',
 'all',
 'that',
 'unusual',
 'my',
 'home',
 'address',
 'is',
 'po',
 'box',
 'that',
 'friend',
 'checks',
 'weekly',
 'and',
 'that',
 'might',
 'get',
 'to',
 'check',
 'every',
 'six',
 'months',
 'or',
 'so',
 'live',
 'in',
 'my',
 'truck',
 'and',
 'at',
 'truck',
 'stops',
 'if',
 'you',
 'need',
 'to',
 'make',
 'patch',
 'available',
 'on',
 'an',
